このコード内のプロセス
テキストからmecabで形態素解析
解析後のtokenからentityのaliasを抽出
aliasー＞entityにマッピング
抽出したentityと病名とentityの解析結果を比較して病名候補を出力

In [ ]:
import pandas as pd
import pyspark
from pyspark.sql.functions import col, countDistinct, when, broadcast, count, row_number, substring,udf,dense_rank,format_string,first,lit, to_date, transform,collect_set
from pyspark.sql.window import Window
import unicodedata
from pyspark.sql.types import StringType
import numpy as np
import pyspark.sql.functions as F
from typing import Optional, List, Tuple
from pyspark.sql import SparkSession


# Sparkセッションがまだ存在しない場合に作成
spark = SparkSession.builder \
    .appName("LoadParquetFile") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.sql.shuffle.partitions", "500") \
    .config("spark.sql.files.maxPartitionBytes", "32m") \
    .getOrCreate()


alias2entity_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/KG/cytoscape/alias2entity.csv"
dis_entity_data_path ="/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/top_aliases_by_main_name_raw_grouped.parquet"

# --- MeCab helpers（そのまま利用） ---
import MeCab
MECAB_DIC_PATH = "/opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd"
_mecab_tagger = None  # lazy singleton

def _get_tagger() -> MeCab.Tagger:
    global _mecab_tagger
    if _mecab_tagger is None:
        _mecab_tagger = MeCab.Tagger(f"-Owakati -d {MECAB_DIC_PATH}")
    return _mecab_tagger

def mecab_tokenize(text: Optional[str]) -> List[str]:
    """Return wakati tokens for a single string without using Spark."""
    if not text:
        return []
    try:
        parsed = _get_tagger().parse(text)
        return parsed.strip().split() if parsed else []
    except Exception:
        return []


def extract_disease_candidates_from_text_weighted(
    text: str,
    spark: SparkSession,
    *,
    # 既存オブジェクトorパス（どちらかでOK）
    alias_df: Optional[pd.DataFrame] = None,
    alias2entity_path: Optional[str] = None,
    dis_entity_df=None,
    dis_entity_data_path: Optional[str] = None,
    # ハイパーパラメータ（disease_countを強め気味の初期値）
    rank_decay: float = 0.85,
    kg_boost: float = 1.15,
    beta: float = 2.0,     # prior寄与の線形強度
    k1: float = 60.0,      # priorのスケール
    gamma_prior: float = 1.5,  # prior寄与の非線形ブースト(>1で強化)
) -> Tuple:
    """
    テキストをMeCabで分かち→alias→diseaseに写像し、重み付きスコアで候補疾患を返す。

    Returns:
        (candidates_df, mapped_entities, tokens)

        candidates_df: Spark DataFrame
            columns: disease, final_score, match_score, prior, match_count,
                     matches, alias_total_cnt, disease_count
        mapped_entities: List[str]  # テキストから得たエンティティ（重複除去済）
        tokens: List[str]           # MeCab分かち書きトークン
    """
    # --- 入力ロード ---
    if alias_df is None:
        if not alias2entity_path:
            raise ValueError("alias2entity_path か alias_df のいずれかを指定してください。")
        alias_df = pd.read_csv(alias2entity_path)

    if dis_entity_df is None:
        if not dis_entity_data_path:
            raise ValueError("dis_entity_data_path か dis_entity_df のいずれかを指定してください。")
        dis_entity_df = spark.read.parquet(dis_entity_data_path)

    # --- tokenize → alias→entity マッピング ---
    tokens = mecab_tokenize(text)

    alias_to_entity = (
        alias_df.dropna(subset=["alias", "Name_mapped"])
                .drop_duplicates(subset=["alias"])
                .set_index("alias")["Name_mapped"]
                .to_dict()
    )

    mapped_entities = [alias_to_entity[t] for t in tokens if t in alias_to_entity]
    mapped_entities = list(dict.fromkeys(mapped_entities))  # 重複除去＆順序保持

    # マッチ対象が空なら空DF返す
    if not mapped_entities:
        schema = """
            disease string,
            final_score double,
            match_score double,
            prior double,
            match_count int,
            matches array<string>,
            alias_total_cnt long,
            disease_count long
        """
        empty_df = spark.createDataFrame([], schema=schema)
        return empty_df, mapped_entities, tokens

    mapped_lit = F.array(*[F.lit(x) for x in mapped_entities])

    # --- 一致抽出 ---
    df = (
        dis_entity_df
        .withColumn("alias_arr", F.expr("transform(aliases_ranked, x -> x.alias)"))
        .withColumn("matches", F.array_intersect(F.col("alias_arr"), mapped_lit))
        .withColumn("match_count", F.size("matches"))
        .filter(F.col("match_count") > 0)
    )

    # --- rank×KG 重み：UDFなしaggregateで match_score ---
    expr_match_score = f"""
    aggregate(
      aliases_ranked,
      0D,
      (acc, x) ->
        acc + IF(
                array_contains(matches, x.alias),
                pow({rank_decay}, x.rank - 1) * IF(x.KG_flag, {kg_boost}, 1.0),
                0D
              )
    )
    """
    df = df.withColumn("match_score", F.expr(expr_match_score))

    # --- disease_count prior と最終スコア（γで非線形強化） ---
    df = (
        df.withColumn(
            "prior",
            (F.col("disease_count") / (F.lit(k1) + F.col("disease_count"))).cast("double")
        )
        .withColumn(
            "final_score",
            F.col("match_score") * F.pow((1 + F.lit(beta) * F.col("prior")), F.lit(gamma_prior))
        )
    )

    candidates = (
        df.select(
            "disease","final_score","match_score","prior","match_count",
            "matches","alias_total_cnt","disease_count"
        )
        .orderBy(
            F.col("final_score").desc(),
            F.col("match_count").desc(),
            F.col("alias_total_cnt").desc_nulls_last()
        )
    )

    return candidates, mapped_entities, tokens


25/11/14 15:44:01 WARN Utils: Your hostname, Donuts-NM3044-2508.local resolves to a loopback address: 127.0.0.1; using 10.10.35.188 instead (on interface en0)
25/11/14 15:44:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/14 15:44:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/14 15:44:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/14 15:44:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
# どちらか：パスで渡す


sample_text ="""
医療DX加算　2025･6算定分動脈硬化性疾患発症予測・脂質管理目標設定アプリ
（医師・医療従事者向け）
一次予防　高リスク
まず生活習慣の改善を行った後、薬物療法の適用を考慮する
管理目標値
LDL-C	＜	120	mg/dL
Non-HDL-C	＜	150	mg/dL
TG	＜	150	mg/dL(空腹時)
＜	175	mg/dL(随時)
HDL-C	≧	40	mg/dL

血糖、HbA1cを測定する

脂質系の検査を行う

検診の結果を持ってこられましたが、肥満があり、高血圧、高脂血症、糖尿病があります。まずは体重減少に努めていただくのが大事で、仕事は介護職という事で体を動かしているようですので、そうすると毎日100Kcal減量していただくというのが基本だろうと思います。それで1ヶ月ほど様子をみせていただき、今日のところは血糖とコレステロールも測らせていただきましょう。家庭血圧も測っていただき、血圧の結果を見て降圧剤の調整も行いたいと思います。1ヶ月に1回ずつ拝見したいと思います。

コレステロールの方は計算してみると、血糖値が高いためLDLコレステロール120未満という値が出ました。今日もう1回再検してみて、その結果をもってコレステロールの薬を考えたいと思います。家庭血圧測定をしていただき、今日は血糖値についての薬を出しましょう。
以上中安ひ記載

（以下幾田記載）
血圧測定指導、100Kcal減量指導（外食編）を行った。
（以上幾田記載）


"""
cands, ents, toks = extract_disease_candidates_from_text_weighted(
    text=sample_text,
    spark=spark,
    alias2entity_path=alias2entity_path,
    dis_entity_data_path=dis_entity_data_path,
    # 調整ノブ（必要なら）
    beta=2.0, k1=40.0, gamma_prior=2.0, rank_decay=0.85, kg_boost=1.15
)

cands.show(50,truncate=False)
print(ents)
print(toks)


+------------------------------+------------------+------------------+------------------+-----------+----------------------------------------+---------------+-------------+
|disease                       |final_score       |match_score       |prior             |match_count|matches                                 |alias_total_cnt|disease_count|
+------------------------------+------------------+------------------+------------------+-----------+----------------------------------------+---------------+-------------+
|内頚動脈狭窄症                |18.275578567925482|2.0983872435440425|0.9755799755799756|4          |[高血圧, 糖尿病, 動脈硬化症, 高脂質血症]|736            |1598         |
|頚動脈硬化症                  |17.598255888338755|2.0166666757256833|0.9770246984491672|3          |[高血圧, 動脈硬化症, 糖尿病]            |2081           |1701         |
|高脂血症                      |17.360666727835465|1.9338299384496538|0.9981112475210123|4          |[高脂質血症, 高血圧, 糖尿病, 検診]      |5510           |21138        |
|２型糖尿病             